In [ ]:
import os
import pandas as pd
import numpy as np
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, roc_curve
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
csv_dir = '../../SEC Trackman Data/'
all_files = [os.path.join(csv_dir, f) for f in os.listdir(csv_dir) if f.endswith('.csv')]

df_list = []

for file in all_files:
    df = pd.read_csv(file)
    df_list.append(df)

all_pitches = pd.concat(df_list, ignore_index=True)

In [ ]:
tagged_pitch_types = all_pitches['TaggedPitchType'].unique()

auto_pitch_types = all_pitches['AutoPitchType'].unique()

In [ ]:
#Get all offspeed pitches
off_speed = all_pitches[all_pitches['TaggedPitchType'].isin(['ChangeUp','Splitter'])]

In [ ]:
#Whiffs
off_speed['whiff'] = np.where(off_speed['PitchCall']=='StrikeSwinging',1,0)

In [ ]:
os_preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','PlateLocHeight','PlateLocSide','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']

In [ ]:
off_speed_X = off_speed[os_preds]
off_speed_y = off_speed['whiff']

In [ ]:
#Train Model
X_train, X_test, y_train, y_test = train_test_split(off_speed_X, off_speed_y, test_size=.33, random_state=25)

model = XGBClassifier()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]

accuracy = accuracy_score(y_test, predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

In [ ]:
cm = confusion_matrix(y_test, predictions)
plt.figure(figsize=(10, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
all_preds = model.predict_proba(off_speed_X)[:,1]
mean_pred = np.mean(all_preds)

print(mean_pred)

sample = off_speed_X.sample(n=10)

sample_preds = model.predict_proba(sample)[:,1]

stuff = sample_preds/mean_pred

print(stuff*100)

In [ ]:
#Auburn Subset
aub_os = off_speed[off_speed['PitcherTeam']=='AUB_TIG']

In [ ]:
# Get unique values
unique_values = off_speed['Pitcher'].unique()
aub_unique_values = aub_os['Pitcher'].unique()

#Print list of Auburn pitchers for selected split
print(aub_unique_values)

# Create a dictionary to store subsets
subsets = {value: off_speed[off_speed['Pitcher'] == value] for value in unique_values}
aub_subsets = {value: aub_os[aub_os['Pitcher'] == value] for value in aub_unique_values}

In [ ]:
def aub_os_stuff(name):
    
    os_preds = ['RelSpeed','VertRelAngle','HorzRelAngle','SpinRate','SpinAxis','RelHeight','RelSide','Extension','VertBreak','InducedVertBreak','HorzBreak','PlateLocHeight','PlateLocSide','ZoneSpeed','VertApprAngle','HorzApprAngle','ZoneTime','pfxx','pfxz','x0','y0','z0','vx0','vy0','vz0','ax0','ay0','az0','SpeedDrop','PitchTrajectoryXc1','PitchTrajectoryXc2','PitchTrajectoryYc0','PitchTrajectoryYc1','PitchTrajectoryYc2','PitchTrajectoryZc0','PitchTrajectoryZc1','PitchTrajectoryZc2']
    
    all_preds = model.predict_proba(off_speed_X)[:,1]
    mean_pred = np.mean(all_preds)

    pitcher = aub_subsets[name]
    pitcher_x = pitcher[os_preds]

    whiff_preds = model.predict_proba(pitcher_x)[:,1]

    stuff = whiff_preds/mean_pred
    
    print(f"{name} Off-speed stuff+: {np.mean(stuff * 100)}")

In [ ]:
#Stuff+ for selected Auburn pitchers Offspeed
for pitcher in aub_unique_values:
    aub_os_stuff(pitcher)

In [ ]:
j_a = subsets['Graves, Griffin']
j_a['TaggedPitchType'].unique()

In [ ]:
c_allsup = subsets['Graves, Griffin']
c_allsup_X = c_allsup[os_preds]

whiff_preds = model.predict_proba(c_allsup_X)[:,1]

stuff = whiff_preds/mean_pred

print(f"Chase Allsup off-speed stuff+:{np.mean(stuff*100)}")

In [ ]:
b_schorr = subsets['Schorr, Ben']
b_schorr_X = b_schorr[os_preds]

whiff_preds = model.predict_proba(b_schorr_X)[:,1]

stuff = whiff_preds/mean_pred

print(f"Ben Schorr off-speed stuff+:{np.mean(stuff*100)}")